In [1]:
# import Packages:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import os, sys

# Get the current working directory
cwd = os.getcwd()
mydir = os.path.abspath(os.path.join(cwd, ".."))
sys.path.append(mydir)
from DataGenerator import genTrainValidFolds

In [2]:
STDbatch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Classification-1/STD/batchData/"
EDGbatch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Classification-1/EDG/batchData/"
HOGp1batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Classification-1/HOGp1/batchData/"

In [20]:
# def toOneHotVector():
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()
    
    
def reshape_data(dataset, labels, featureSize, numLabels, sample_size=None):
    if sample_size:
        dataset = dataset[:sample_size].reshape(sample_size, featureSize) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    else:
        dataset = dataset.reshape(len(dataset), featureSize) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    return dataset, labels

In [74]:
class BuildNeuralNet():
    
    def __init__(self):
        
        seed = 2316
        self.featureSize = 162
        self.numHidden1 = 1000
        self.numHidden2 = 1000
        self.numHidden3 = 1000
        self.numLabels = 2
        self.alpha = 0.02
        self.momentum = 0.9
        
        self.weights = {
            "inp_to_hid1_wght": tf.Variable(tf.random_normal([self.featureSize, self.numHidden1], seed=seed)),
            "hid1_to_hid2_wght" : tf.Variable(tf.random_normal([self.numHidden1, self.numHidden2], seed=seed)),
            "hid2_to_hid3_wght": tf.Variable(tf.random_normal([self.numHidden2, self.numHidden3], seed=seed)),
            "hid3_to_out_wght": tf.Variable(tf.random_normal([self.numHidden3, self.numLabels], seed=seed))
        }
        
        self.biases = {
            "hid1_bias": tf.Variable(tf.random_normal([self.numHidden1], seed=seed)),
            "hid2_bias": tf.Variable(tf.random_normal([self.numHidden2], seed=seed)),
            "hid3_bias": tf.Variable(tf.random_normal([self.numHidden3], seed=seed)),
            "out_bias": tf.Variable(tf.random_normal([self.numLabels], seed=seed))
        }
        
        
    def trainNet(self):
        trainData = tf.placeholder(tf.float32, [None, self.featureSize])
        trainLabels = tf.placeholder(tf.float32, [None, self.numLabels])

        # 1st Hidden Layer State
        inp_to_hid1 = tf.matmul(trainData, self.weights['inp_to_hid1_wght']) + self.biases['hid1_bias']
        hid1State = tf.sigmoid(inp_to_hid1)
        
        # 2nd Hidden Layer State
        hid1_to_hid2 = tf.matmul(hid1State, self.weights['hid1_to_hid2_wght']) + self.biases['hid2_bias']
        hid2State = tf.sigmoid(hid1_to_hid2)
        
        # 3nd Hidden Layer State
        hid2_to_hid3 = tf.matmul(hid2State, self.weights['hid2_to_hid3_wght']) + self.biases['hid3_bias']
        hid3State = tf.sigmoid(hid2_to_hid3)
        
        # Output Layer State
        hid3_to_out = tf.matmul(hid3State, self.weights['hid3_to_out_wght']) + self.biases['out_bias']
        outState = tf.nn.softmax(hid3_to_out)
        
        # Loss Function and Optimization
        lossCE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hid3_to_out, labels=trainLabels))
#         optimizer = tf.train.AdamOptimizer(learning_rate=self.alpha).minimize(lossCE)
        optimizer = tf.train.MomentumOptimizer(self.alpha, 
                                            self.momentum, 
                                            use_locking=False, 
                                            name='Momentum', 
                                            use_nesterov=True).minimize(lossCE)
        
        # Evaluate model
        trainPred = tf.equal(tf.argmax(outState, 1), tf.argmax(trainLabels, 1))
        trainAccuracy = tf.reduce_mean(tf.cast(trainPred, tf.float32))
        
        return dict(
            trainData = trainData, 
            trainLabels = trainLabels,
            weightsLRND = self.weights,
            biasesLRND = self.biases,
            optimizer = optimizer,
            lossCE = lossCE,
            accuracy = trainAccuracy
        )
    
    
    def crossValid(self):
            validData = tf.placeholder(tf.float32, [None, self.featureSize])
            validLabels = tf.placeholder(tf.float32, [None, self.numLabels])

            # 1st Hidden Layer State
            valid_hid1State = tf.sigmoid(tf.matmul(validData, self.weights['inp_to_hid1_wght']) + self.biases['hid1_bias'])

            # 2nd Hidden Layer State
            valid_hid2State = tf.sigmoid(tf.matmul(valid_hid1State, self.weights['hid1_to_hid2_wght']) + self.biases['hid2_bias'])

            # 3nd Hidden Layer State
            valid_hid3State = tf.sigmoid(tf.matmul(valid_hid2State, self.weights['hid2_to_hid3_wght']) + self.biases['hid3_bias'])

            # Output Layer State
            validOutState = tf.nn.softmax(tf.matmul(valid_hid3State, self.weights['hid3_to_out_wght']) + self.biases['out_bias'])

            validPred = tf.equal(tf.argmax(validOutState, 1), tf.argmax(validLabels, 1))
            validAccuracy = tf.reduce_mean(tf.cast(validPred, tf.float32))
            
            return dict(
                validData = validData,
                validLabels = validLabels,
#                 validwghts = self.weights,
#                 validbias = self.biases,
#                 validPred = validPred,
                validAccuracy = validAccuracy
            )
            
            

In [81]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


epochs = 30
def model(trainGraphDict, validGraphDict):
    print (trainGraphDict)
    print ('')
    print (validGraphDict)
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for foldNUM, (trainDataIN, trainLabelsIN, validDataIN, validLabelsIN, labelDict) in enumerate(genTrainValidFolds(HOGp1batch_dir, oneHot=True)):
            print ('Validation Data and Labels shape: ', validDataIN.shape, validLabelsIN.shape)
            print ('Training Data and Labels shape: ', trainDataIN.shape, trainLabelsIN.shape)
            print ('The Label Dictionary is given as: ', labelDict)

            for epoch in range(epochs): 
                feed_dict = {trainGraphDict['trainData']: trainDataIN,
                             trainGraphDict['trainLabels']: trainLabelsIN
                        }
                
                
                _, loss, tacc, twghts, tbiases = sess.run([trainGraphDict['optimizer'], 
                                                        trainGraphDict['lossCE'], 
                                                        trainGraphDict['accuracy'],
                                                        trainGraphDict['weightsLRND'],
                                                        trainGraphDict['biasesLRND']], feed_dict=feed_dict)

                print ("Fold: " + str(foldNUM+1) + ", Iter: " + str(epoch) + ", Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(tacc))

                # During the Last iteration of the nth fold batch, we should calculate the model accuracy with Cross Validataion Data
                if epoch==epochs-1:
                    print ('')

                    feed_dict = {validGraphDict['validData']: validDataIN,
                             validGraphDict['validLabels']: validLabelsIN
                        }
                
                    vacc = sess.run(validGraphDict['validAccuracy'], feed_dict=feed_dict)
                    print ("Fold: " + str(foldNUM+1) + ", Cross Validation Accuracy= " + "{:.5f}".format(vacc))
                    
            print ('')
            print ('')

            break
#             epoch += 1
#         print ("Optimization Finished!")
#     obj_SVM = Models()
#     batchEvalDict[foldNUM] = obj_SVM.classify(trainData, trainLabels, validData)

reset_graph()
objNNET = BuildNeuralNet()
trainGraphDict = objNNET.trainNet()
validGraphDict = objNNET.crossValid()
model(trainGraphDict, validGraphDict)

{'trainLabels': <tf.Tensor 'Placeholder_1:0' shape=(?, 2) dtype=float32>, 'weightsLRND': {'inp_to_hid1_wght': <tensorflow.python.ops.variables.Variable object at 0x11d3b6400>, 'hid3_to_out_wght': <tensorflow.python.ops.variables.Variable object at 0x11e2c5f28>, 'hid2_to_hid3_wght': <tensorflow.python.ops.variables.Variable object at 0x11d851f60>, 'hid1_to_hid2_wght': <tensorflow.python.ops.variables.Variable object at 0x10ac82390>}, 'biasesLRND': {'hid3_bias': <tensorflow.python.ops.variables.Variable object at 0x11e2c55c0>, 'out_bias': <tensorflow.python.ops.variables.Variable object at 0x11e2c5358>, 'hid2_bias': <tensorflow.python.ops.variables.Variable object at 0x11d6dca20>, 'hid1_bias': <tensorflow.python.ops.variables.Variable object at 0x11e2c5390>}, 'accuracy': <tf.Tensor 'Mean_1:0' shape=() dtype=float32>, 'lossCE': <tf.Tensor 'Mean:0' shape=() dtype=float32>, 'trainData': <tf.Tensor 'Placeholder:0' shape=(?, 162) dtype=float32>, 'optimizer': <tensorflow.python.framework.ops.O